In [12]:
import numpy as np
from numpy import linalg as LA
import csv
import matplotlib.pyplot as plt
from Library import ka, Create_MPO, Create_random_MPS, Pauli, Model_coefficients, i_trunc, Entropy

data_type=np.complex128

interval = 30
d = 2
L = 80        # system size
n = 3000   # Number of zips
h_x, h_y, h_z = 1., 0.9, 1.  # Field in the different directions
J_xx, J_yy, J_zz = 1., 0.2, 1.  # Strength of spin interactions
Model = [["x"],["z"]]     # The first part of the model gives us the spin interaction and the second one the fields
chi_MPO = len(Model[0]) + 2 
t = 8
chim = [4]
Entropiq = np.zeros(interval)
Energia = np.zeros(interval)

S_z, S_x, S_y, S1 = Pauli()
Interactions, Fields = Model_coefficients(J_xx,J_yy,J_zz,h_x,h_y,h_z)
 
def H_mult(v):
    return np.matmul(H,v)

def Compue_Energy_iTEBD(L_env,Upper,W,R_env):
    Ener = np.einsum("amc,aikb->ikbmc",L_env,Upper)
    Ener = np.einsum("ikbmc,mlij->ljkbc",Ener,W)
    Ener = np.einsum("ljkbc,ldkv->jdbvc",Ener,W)
    Ener = np.einsum("jdbvc,cjvh->dbh",Ener,np.conj(Upper))
    Ener = np.einsum("dbh,bdh->",Ener,R_env)
    return Ener


In [13]:
from numpy.random import random as rnd
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import eigsh
import cProfile
import pstats
np.random.seed(121)

gap = 0.01
print("start")
for ty in chim:
 chi_max = ty  
 chis = ka(L,chi_max)
 X, V = Create_random_MPS(L,chis)
 v_L,v_R=np.zeros(chi_MPO,dtype=data_type),np.zeros(chi_MPO,dtype=data_type)
 v_L[0],v_R[-1]=1.,1. 

 L_env,R_env=np.zeros((chi_max,chi_MPO,chi_max),dtype=data_type),np.zeros((chi_max,chi_MPO,chi_max),dtype=data_type)

 L_env=np.einsum("ab,c->acb",np.eye(chi_max,dtype=data_type),v_L)         # Initial Righ and left environments
 R_env=np.einsum("ab,c->acb",np.eye(chi_max,dtype=data_type),v_R)

 Gamma=[X[L//2],X[L//2+1]]
 lam=[V[L//2],V[L//2+1]]

 A = np.einsum("a,aib->aib",lam[0],Gamma[0])
 B = np.einsum("aib,b->aib",Gamma[1],lam[0])

 Entropiq = np.zeros(interval)
 Energia = np.zeros(interval)

 for ja in range(interval):
  print(ja)
  h_z = ja/10+gap
  L_env=np.einsum("ab,c->acb",np.eye(chi_max,dtype=data_type),v_L)         # Initial Righ and left environments
  R_env=np.einsum("ab,c->acb",np.eye(chi_max,dtype=data_type),v_R)

  Interactions, Fields = Model_coefficients(J_xx,J_yy,J_zz,h_x,h_y,h_z)
  W = Create_MPO(Model,S1,Interactions,Fields,chi_MPO, data_type)
  en = 0.
  for i in range(n):
    # Update Hamiltonian
    if i!=0:
     en = w[0]
    H=np.einsum("abc,bdij->adijc",L_env,W)
    H=np.einsum("adijc,dekl->aekl ijc",H,W)
    H=np.einsum("aekl ijc,feg->aikf cjlg",H,R_env).reshape((chi_max*chi_max*d**2,chi_max*chi_max*d**2))
    
    As = LinearOperator((chi_max*chi_max*d**2,chi_max*chi_max*d**2), matvec=H_mult,dtype=data_type)
    
    Theta = np.einsum("aib,b->aib",A,lam[1])
    Theta = np.einsum("aib,bjc->aijc",Theta,B).reshape(chi_max*chi_max*d**2)
    
    # Find ground state
    w, v = eigsh(As,k=1,v0=Theta,which="SA",maxiter=400000)
    dn = w[0] 
    Theta_p = v[:,0].reshape(chi_max*d,chi_max*d)
    Theta_p/=LA.norm(Theta_p)
    # Truncate
    A,lamb,B=i_trunc(chi_max,Theta_p,d=d)
    lam[1]=lamb

    #Updating the left and right environments
    L_env=np.einsum("amb,aic->mbic",L_env,A)   
    L_env=np.einsum("mbic,mlij->bclj",L_env,W)   
    L_env=np.einsum("bclj,bjd->cld",L_env,np.conj(A))

    R_env=np.einsum("amb,cia->mbci",R_env,B)
    R_env=np.einsum("mbci,lmij->bclj",R_env,W)
    R_env=np.einsum("bclj,djb->cld",R_env,np.conj(B))
    
    #f i>1:
    

    # Compute energy
  print(en/(n-1)-dn/(n))
  # 2-site wave function
  Upper=np.einsum("aib,b,bjc->aijc",A,lam[1],B)

 # Energy
  Energia[ja] = Compue_Energy_iTEBD(L_env,Upper,W,R_env).real/2/(i+1)
  Entropiq[ja] = Entropy(lam[1])
 
 File_name = 'Data_iMPS/Data__iMPS_chi_max'+ str(chi_max)+".csv"
 with open(File_name, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Entropy, chi_max=",chi_max])
    writer.writerow(np.arange(interval)/10+gap)
    writer.writerow(Entropiq)

    writer.writerow(["Energy, chi_max=",chi_max])
    writer.writerow(np.arange(interval)/10+gap)
    writer.writerow(Energia)


start
0
1.1145332012851838e-07
1
2.346073255665715e-07
2
1.5783612594688634e-07
3
2.492093158501518e-07
4
5.346795672878102e-07
5
1.8497204035128334e-07
6
1.0680142588093133e-06
7
4.6324614766390937e-07
8
1.216857555164097e-06
9
1.908382233573036e-06
10
-6.988084066961164e-08
11
-6.425584766489578e-07
12
-2.5454408527636474e-06
13
-9.187322058235026e-06
14
3.27923114165074e-05
15
3.931523075717536e-05
16
-1.148854829713386e-05
17
3.428416346462626e-05
18
9.39719534005512e-05
19
5.219125364108734e-05
20
0.00011793402740112313
21
0.0001327102053618745
22
0.00013134460601449405
23
-0.00033111796585938436
24
-0.0002773944629295144
25
0.00012819365351113277
26
-0.0001395054281800867
27
0.00010464550613242807
28
8.533330013538176e-05
29
9.413954151060722e-05


In [49]:
import csv
x = np.arange(0.1,30.1,1)
#plt.plot(x,Entropiq)
#print(Energia)
gap = 0.1
print(lam[1])



[9.96095610e-01 8.82782369e-02 6.96260871e-04 6.17056050e-05
 5.49853816e-06 4.87314788e-07 4.26732039e-08 3.84888421e-09
 3.78180881e-09 3.65185311e-10]
